# DecisionTree Assignment - 20기 박준

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [2]:
import pandas as pd 
import numpy as np

In [3]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [4]:
len(pd_data['age'])

14

In [5]:
def get_gini(df, label):
    counts = df[label].value_counts()
    length=len(df[label])
    gini=1
    for i in counts:
        gini-=(i/length)**2
    
    
    return gini

In [6]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [7]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    length=len(uniques)
    start=1
    while (True):
        combis = list(combinations(uniques, start)) #combination하고 이것을 이제 리스트에 저장. 
        start+=1
        for i in combis:
            result.append(i)
        length-=2
        if (length<2):
            break

    
    return result

In [8]:
get_binary_split(pd_data,'age') #한 리스트 안에 한개의 키와 그 외 나머지 튜플로 묶은 것.
#굳이 A,B,C가 있는데 A를 했을 때, B,C를 따로 구해줄 필요 없이 그냥 하나만 구해주었다. 

[('youth',), ('middle_aged',), ('senior',)]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [29]:
l=('youth','senior')
new=pd.DataFrame(columns=pd_data.columns)
for i in l:
    cur_df=pd_data.loc[pd_data['age'] == i]
    new=pd.concat([new,cur_df],axis=0)
get_gini(new,'class_buys_computer')

0.5

In [35]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    for i in binary_split:
        new_df1=pd.DataFrame(columns=df.columns)
        new_df2=df
        for j in i:
            cur_df=df.loc[df[attribute] == j]
            new_df1=pd.concat([new_df1, cur_df], axis = 0)
            new_df2=new_df2.loc[new_df2[attribute] != j]   #new_df1에는 split한 key값이 포함되어 있는 df, df2에는 반대로 key값 말고 나머지 것들만 포함되어 있는 df
        result[i]=get_gini(new_df1,label)*(len(new_df1)/len(df))+get_gini(new_df2,label)*(len(new_df2)/len(df))   #공식으로 값 구한후, 대입.
     
    
    return result

In [37]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [38]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [39]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [45]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])
result={}
# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    cur=min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())[1]
    result[feature]=cur
result


{'age': 0.35714285714285715,
 'income': 0.4428571428571429,
 'student': 0.3673469387755103,
 'credit_rating': 0.42857142857142855}

gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [46]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [54]:
from math import log2

def getEntropy(df, feature) :
    counts = df[label].value_counts()
    result=0
    for count in counts:
        p=count/len(df)
        result-=p*log2(p)

    return result
    


In [55]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [66]:
keys=pd_da

['income', 'student', 'credit_rating', 'class_buys_computer']

In [71]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    attributes = list(pd_data.columns[pd_data.columns != feature]) #이제 특성값들이 list안에 정해져있음.    
    result = {}
    for attr in attributes:
        attr_value=getEntropy(df,feature)
        keys=df[attr].unique()
        for key in keys:
            cur_df=df.loc[df[attr] == key]
            attr_value-=getEntropy(cur_df,feature)*len(cur_df)/len(df)
        result[attr]=attr_value

    

    return result
    

In [72]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443927,
 'income': 0.029222565658954813,
 'student': 0.15183550136234159,
 'credit_rating': 0.048127030408269544}

각자 attribute에 대하여 어떻게 gini 계산값을 계산하고 무엇을 고를지 선택.
1. refund 
6/10(1-(4/6)^2-(2/6)^2)+4/10(1-(0)^2-(1)^2) = 0.267

2. Marital Status
6/10(1-(4/6)^2-(2/6)^2)+4/10(1-(2/4)^2-(2/4)^2) = 0.467

3. Taxable Income 
7/10(1-(4/7)^2-(3/7)^2)+3/10(1-(0)^2-(1)^2) = 0.343

결국에 가장 낮은 1이 선택된다.
이 경우에는 모든 attribute의 값들이 2개 뿐이므로 이진분류되는 경우가 하나뿐이므로 그냥 바로 구하면 된다.

